# Table of Contents
 <p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

Next, try with graph_tool:

In [1]:
from graph_tool.all import *
import csv

hy = Graph()
v_name = hy.new_vertex_property("string")
v_color = hy.new_vertex_property("string")
v_size = hy.new_vertex_property("float")
# v_text_offset = hy.new_vertex_property('vector<double>')

# The file nodes.tsv contains the hierarchy of nodes, from a single "Origin" origin, in "from"-"to" relations
with open('nodes.tsv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter='\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        q  = row[0]       # from
        qc = row[1]       # from_color
        z  = row[2]       # to
        zc = row[3]       # to_color
        if len(graph_tool.util.find_vertex(hy, v_name, q)) == 0:
            s = hy.add_vertex()
            v_name[s] = q
            v_color[s] = qc
            # v_text_offset[s] = [0.0, 0.0]
            if qc == "black": v_size[s] = 0
            else: v_size[s] = 10
        else:
            s = graph_tool.util.find_vertex(hy, v_name, q)[0]

        if len(graph_tool.util.find_vertex(hy, v_name, z)) == 0:
            t = hy.add_vertex()
            v_name[t] = z
            v_color[t] = zc
            # v_text_offset[s] = [0.0, 0.0]
            if zc == "black": v_size[t] = 0
            else: v_size[t] = 10
        else:
            t = graph_tool.util.find_vertex(hy, v_name, z)[0]
        
        if not hy.edge(s, t):
            hy.add_edge(s, t)

pos = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

graph_draw(hy, pos=pos, vertex_fill_color=v_color,                                    # vertex drawing
           vertex_text=v_name, vertex_font_size=10, vertex_text_position="centered",  # vertex text
            output_size=(1024, 768), edge_pen_width=2, output="hierarchy.png")

if graph_tool.util.find_vertex(hy, "in", 0):
    l = graph_tool.util.find_vertex(hy, "in", 0)
    for n in l:
        print("Possible indegree problem with vertex {}".format(v_name[n]))

hy.save("hierarchy.xml.gz")

Possible indegree problem with vertex Origin


In [2]:
from matplotlib import colors

nw = Graph()
nwv_name     = nw.new_vertex_property("string")
nwv_color    = nw.new_vertex_property("string")
nwv_size_1   = nw.new_vertex_property("float")
nwv_size_2   = nw.new_vertex_property("float")
nwe_color    = nw.new_edge_property("string")
nwe_gradient = nw.new_edge_property('vector<double>')
# we want these for masking later:
nwv_1923     = nw.new_vertex_property("bool")
nwv_1936     = nw.new_vertex_property("bool")
nwv_1923_2   = nw.new_vertex_property("bool")
nwv_1936_128 = nw.new_vertex_property("bool")
nwe_1923     = nw.new_edge_property("bool")
nwe_1936     = nw.new_edge_property("bool")
nwe_1923_2   = nw.new_edge_property("bool")
nwe_1936_128 = nw.new_edge_property("bool")


# g.edge_properties['egradient'] = egradient

for v in hy.vertices():
    n = nw.add_vertex()
    nwv_name[n]  = v_name[v]
    nwv_color[n] = v_color[v]
    nwv_size_1[n]  = v_size[v]
    nwv_size_2[n]  = 3
    
    if nwv_name[n][:10] == "Draft 1923":
        nwv_1923[n] = True
    else:
        nwv_1923[n] = False

    if nwv_name[n][:10] == "Draft 1936":
        nwv_1936[n] = True
    else:
        nwv_1936[n] = False
    
    if nwv_name[n] == "Draft 1923_1923/2 (1)":
        nwv_1923_2[n] = True
    else:
        nwv_1923_2[n] = False
    
    if nwv_name[n] == "Draft 1936_1936/128":
        nwv_1936_128[n] = True
    else:
        nwv_1936_128[n] = False


with open('edges.tsv') as csvfile:
    count = 0
    readCSV = csv.reader(csvfile, delimiter = '\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        source = ""
        target = ""
        
        f = row[0]       # from
        t = row[1]       # to
    
        if len(graph_tool.util.find_vertex(nw, nwv_name, f)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a source in edges list.".format(f))
            break
        else:
            source = graph_tool.util.find_vertex(nw, nwv_name, f)[0]
        
        if len(graph_tool.util.find_vertex(nw, nwv_name, t)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a target in edges list.".format(t))
            break
        else:
            target = graph_tool.util.find_vertex(nw, nwv_name, t)[0]
        
        e = nw.add_edge(source, target)
        count += 1
        nwe_color[e] = nwv_color[source]
        estartcolor = [0] + list(colors.to_rgba(nwv_color[target]))
        eendcolor = [1] + list(colors.to_rgba(nwv_color[source]))
        nwe_gradient[e] = estartcolor + eendcolor
        
        print(str(count) + ": Create edge from '" + f + "' to '" + t + "'.")

        if nwv_name[source][:10] == "Draft 1923":
            nwv_1923[target] = True
            nwe_1923[e]      = True
        else:
            nwv_1923[n] = False
            nwe_1923[e] = False

        if nwv_name[source][:10] == "Draft 1936":
            nwv_1936[target] = True
            nwe_1936[e]      = True
        else:
            nwv_1936[n] = False
            nwe_1936[e] = False

        if nwv_name[source] == "Draft 1923_1923/2 (1)":
            nwv_1923_2[target] = True
            nwe_1923_2[e]      = True
        else:
            nwv_1923_2[n] = False
            nwe_1923_2[e] = False

        if nwv_name[source] ==  "Draft 1936_1936/128":
            nwv_1936_128[target] = True
            nwe_1936_128[e]      = True
        else:
            nwv_1936_128[n] = False
            nwe_1936_128[e] = False


l = graph_tool.util.find_vertex(nw, "total", 0)
for i, n in enumerate(l):
    print(str(i+1) + ": 0-degree vertex: {}".format(v_name[n]))

nw.save("network.xml.gz")

graph_draw(nw, vertex_text=nwv_name, vertex_fill_color=nwv_color, vertex_font_size=10, vertex_text_position=10,
           output_size=(1024, 768), edge_pen_width=2, output="network_1.png")


1: Create edge from 'Draft 1923_1923/1' to 'Germany 1919_Art.151'.
2: Create edge from 'Draft 1923_1923/2 (1)' to 'Book of Mencius'.
3: Create edge from 'Draft 1923_1923/2 (1)' to 'Czechoslovakia 1920_Art.108'.
4: Create edge from 'Draft 1923_1923/2 (1)' to 'Czechoslovakia 1920_Art.109'.
5: Create edge from 'Draft 1923_1923/2 (1)' to 'Far Eastern Republic/Chita 1920'.
6: Create edge from 'Draft 1923_1923/2 (1)' to 'Poland 1921_Art.99'.
7: Create edge from 'Draft 1923_1923/2 (1)' to 'Portugal 1911_Art.25'.
8: Create edge from 'Draft 1923_1923/2 (1)' to 'Pre-Qin Scholars'.
9: Create edge from 'Draft 1923_1923/2 (1)' to 'The Soviet Russia 1918'.
10: Create edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.153'.
11: Create edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.155 (1)'.
12: Create edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.155 (2)'.
13: Create edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.155 (3)'.
14: Create edge from 'Draft 1923_1923/2 (2)' to '

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fb88a73c668, at 0x7fb88a745048>

Now, create an edge-bundling visualization according to:

Holten, Danny: "Hierarchical Edge Bundles: Visualization of Adjacency Relations in Hierarchical Data", in: *IEEE Transactions on Visualization and Computer Graphics* 12/5 (2006), pp. 741–748, DOI: 10.1109/TVCG.2006.147.

In [3]:
# state = graph_tool.inference.minimize.minimize_nested_blockmodel_dl(nw, deg_corr=True)
# t     = graph_tool.inference.nested_blockmodel.get_hierarchy_tree(state)[0]

hypos  = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

cts08   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.8, max_depth=3)
cts06   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.6, max_depth=3)
pos   = nw.own_property(hypos)

l = graph_tool.util.find_vertex(nw, "total", 0)
for n in l:
    print("Removing 0-degree vertex {}".format(v_name[n]))
nw.remove_vertex(l)

# ============== Output for both Drafts ==============

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="both_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="both_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="both_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="both_b08_solid.png")

## ============ Only Draft 1923 ===============
nw.set_vertex_filter(nwv_1923)
nw.set_edge_filter(nwe_1923)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923_b08_solid.png")

nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1936 ===============
nw.set_vertex_filter(nwv_1936)
nw.set_edge_filter(nwe_1936)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1923, Art. 2 (1) ===============
nw.set_vertex_filter(nwv_1923_2)
nw.set_edge_filter(nwe_1923_2)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923-2_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923-2_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923-2_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1923-2_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1936, Art. 128   ===============
nw.set_vertex_filter(nwv_1936_128)
nw.set_edge_filter(nwe_1936_128)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936-128_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936-128_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936-128_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.8, output="1936-128_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

Print("Done.")

Removing 0-degree vertex Origin
Removing 0-degree vertex Draft Articles
Removing 0-degree vertex Foreign Legislation
Removing 0-degree vertex Chinese Sources
Removing 0-degree vertex Chinese Traditions
Removing 0-degree vertex Party‘s Doctrines
Removing 0-degree vertex Austria 1920
Removing 0-degree vertex CM
Removing 0-degree vertex Czechoslovakia 1920
Removing 0-degree vertex Ecuador 1929
Removing 0-degree vertex Estonia 1920
Removing 0-degree vertex Honduras 1924
Removing 0-degree vertex Lithuania 1928
Removing 0-degree vertex Mexico 1917
Removing 0-degree vertex Netherlands 1922
Removing 0-degree vertex Peru 1933
Removing 0-degree vertex Poland 1921
Removing 0-degree vertex Portugal 1911
Removing 0-degree vertex Portugal 1933
Removing 0-degree vertex Romania 1923
Removing 0-degree vertex Spain 1931
Removing 0-degree vertex Switzerland 1874
Removing 0-degree vertex The Covenant of the League of Nations 1920
Removing 0-degree vertex TV
Removing 0-degree vertex Yugoslavia 1931
Removin